<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>
<h3>In this project, I will be explore, segment, and cluster the neighborhoods in the city of Toronto. </h3>
<hr>

Firstly, importing required libararies I am going ot use in this assignment

In [4]:
from bs4 import BeautifulSoup  #library web scrapping
import requests     # library for web URL
import pandas as pd # library for data analsysis
import csv          # library for reading CSV file

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


The data for this assignmen is from a wiki page

In [5]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

Reading the wiki page, just the table class of 'wikitable sortable'
<br>Then insert the data into a dictionary
<br>If a postal code has muntile neighborhood, then concatenate these neighborhoods

In [6]:
source = requests.get(wiki_url).text

soup = BeautifulSoup(source, 'lxml')

neighborhood_dict = {}

for table in soup.find_all('table', class_='wikitable sortable'):
    #print (table.prettify())
    for rows in table.find_all('tr'):
        row_values = rows.text.split('\n')
        postal_code = row_values[1]
        borough = row_values[2]
        neighborhood_name = row_values[3]
        if (borough != 'Not assigned' and borough !='Borough'):
            if (postal_code in neighborhood_dict):
                neighborhood_dict[postal_code] =  neighborhood_dict[postal_code]+','+ neighborhood_name
            else:
                neighborhood_dict[postal_code] = borough + '|' + neighborhood_name

Now reading the Geospatial CSV file to fing the Latitude and Longitude of each Postal Code

In [7]:
#Assigning Latitude and Longitude to each Postal Code from the CSV file
with open('Geospatial_Coordinates.csv', mode ='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
            neighborhood_dict[row["Postal Code"]] = neighborhood_dict[row["Postal Code"]] + "|" + row["Latitude"] + "|" + row["Longitude"]


Now converting the dictionary into a Panda dataframe. If a postal code doesn't have any Neighborhood, then its neighborhood is its borough

In [8]:
column_names =['PostalCode','Borough','Neighborhood','Latitude', 'Longitude']
neighborhoods = pd.DataFrame(columns=column_names)

for pc, val in neighborhood_dict.items():
    vals = val.split('|')
    br = vals[0]     #Borough Value from the dictionary
    ne = vals[1]     #Neighborhood values from the dictionary
    la = vals[2]     #Latitude
    lo = vals[3]     #Longitude
    if (ne == "Not assigned"):
        ne = br
    neighborhoods = neighborhoods.append({'PostalCode':pc,'Borough': br,'Neighborhood': ne,'Latitude':la, 'Longitude':lo,}, ignore_index=True)
neighborhoods.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7532586,-79.3296565
1,M4A,North York,Victoria Village,43.7258823,-79.3155716
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.6542599,-79.3606359
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.4647633
4,M7A,Queen's Park,Queen's Park,43.6623015,-79.3894938


In [11]:
#Retrieving Latitude and Longitude to Toronto, ON
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")  #user_agent= Nominatim use this value ot limit the number of request per applicaiton
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lng)],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto